In [1]:
%tensorflow_version 1.x
%cd /content
!git clone https://github.com/tomyaacov/RPNIComposition
%cd RPNIComposition
!pip install numpy==1.19.5

TensorFlow 1.x selected.
/content
Cloning into 'RPNIComposition'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 61 (delta 21), reused 41 (delta 8), pack-reused 0
Unpacking objects: 100% (61/61), done.
/content/RPNIComposition
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle
import time
import sys
import random
random.seed(1)

In [3]:
with open("data/ABC_10_train.pkl", 'rb') as file:
    train = pickle.load(file)
with open("data/ABC_10_test.pkl", 'rb') as file:
    test = pickle.load(file)

In [4]:
train_data = [list(x[0]) for x in train]
test_data = [list(x[0])  for x in test]
train_labels = np.array([int(x[1]) for x in train])
test_labels = np.array([int(x[1]) for x in test])

In [5]:
DICTIONARY = ["A", "B", "C"]
def get_enc(c):
  return DICTIONARY.index(c)
train_data_enc = [[get_enc(c) for c in d] for d in train_data]
test_data_enc = [[get_enc(c) for c in d] for d in test_data]

In [6]:
max_length = 10
train_data_pad = pad_sequences(train_data_enc,maxlen=max_length,padding='post',value=3)
test_data_pad = pad_sequences(test_data_enc,maxlen=max_length,padding='post',value=3)

In [7]:
model=Sequential()

model.add(Embedding(4,2,input_length=10))
model.add(LSTM(12,input_shape=(10, 2),activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(12,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(12,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 2)             8         
_________________________________________________________________
lstm (LSTM)                  (None, 10, 12)            720       
_________________________________________________________________
dropout (Dropout)            (None, 10, 12)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 12)                1200      
_________________________________________________________________
dropout_1 (Dropout)        

In [8]:
start_time = time.time()
history = model.fit(train_data_pad, train_labels, epochs=3, batch_size=32, validation_data=(test_data_pad, test_labels))
print("lstm:")
print("--- %s seconds training ---" % (time.time() - start_time))
print("lstm acc", history.history["acc"])
lstm_acc = history.history["acc"]
print("lstm val acc", history.history["val_acc"])
lstm_val_acc = history.history["val_acc"]

Train on 62000 samples, validate on 26572 samples
Epoch 1/3
62000/62000 [==============================] - 59s 952us/sample - loss: 0.3837 - acc: 0.8606 - val_loss: 0.3317 - val_acc: 0.8625
Epoch 2/3
62000/62000 [==============================] - 63s 1ms/sample - loss: 0.3245 - acc: 0.8606 - val_loss: 0.3010 - val_acc: 0.8625
Epoch 3/3
62000/62000 [==============================] - 57s 917us/sample - loss: 0.2970 - acc: 0.8606 - val_loss: 0.2748 - val_acc: 0.8625
lstm:
--- 185.48948979377747 seconds training ---
lstm acc [0.8605806, 0.8605806, 0.8605806]
lstm val acc [0.86248684, 0.86248684, 0.86248684]


In [9]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(train_labels, np.ravel((model.predict(train_data_pad) > 0.5)*1)).ravel()
print((tn, fp, fn, tp))

(53356, 0, 8644, 0)


In [11]:
sum(train_labels)

8644

In [10]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(test_labels, np.ravel((model.predict(test_data_pad) > 0.5)*1)).ravel()
print((tn, fp, fn, tp))

(22918, 0, 3654, 0)


In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
import pandas as pd
path = '/content/drive/MyDrive/research/abc_model_history.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df = pd.DataFrame.from_dict(history.history)
  df.to_csv(f)

path = '/content/drive/MyDrive/research/abc_model.h5'
model.save(path)

In [18]:
(np.ravel((model.predict(train_data_pad) > 0.5)*1) == train_labels).sum() / train_labels.size

0.8605806451612903

In [22]:
train_data[np.ravel((model.predict(train_data_pad) > 0.5)*1) == train_labels, :]

TypeError: ignored

In [23]:
misclassified_samples = []
is_accurate = np.ravel((model.predict(train_data_pad) > 0.5)*1) == train_labels
for i,b in enumerate(is_accurate):
  if b:
    misclassified_samples.append(train_data[i])
print(misclassified_samples)

[['B', 'B', 'A', 'A', 'B', 'C', 'A', 'C', 'A', 'A'], ['C', 'C', 'A', 'C', 'C', 'C', 'B', 'C'], ['C', 'C', 'A', 'A', 'A', 'B', 'B', 'B', 'C', 'A'], ['C', 'C', 'B', 'A', 'C', 'C', 'A', 'A', 'A', 'C'], ['B', 'C', 'B', 'B', 'A', 'C', 'A', 'B', 'A', 'B'], ['C', 'A', 'C', 'C', 'B', 'A', 'C', 'B', 'C', 'C'], ['B', 'C', 'A', 'C', 'A', 'A', 'B', 'C', 'A', 'B'], ['A', 'C', 'A', 'C', 'A', 'C', 'A', 'A'], ['C', 'A', 'A', 'C', 'C', 'C', 'B', 'C', 'A', 'C'], ['B', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'A'], ['C', 'A', 'A', 'A', 'A', 'C', 'B', 'C', 'C'], ['A', 'C', 'C', 'C', 'B', 'B', 'A', 'C', 'C', 'B'], ['C', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'A', 'A'], ['B', 'C', 'C', 'C', 'B', 'B', 'C', 'A', 'C', 'B'], ['C', 'A', 'B', 'C', 'B', 'B', 'B', 'A', 'C', 'B'], ['C', 'A', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'C'], ['B', 'C', 'B', 'A', 'A', 'A', 'A', 'C', 'A', 'A'], ['A', 'B', 'B', 'C', 'A', 'C', 'A', 'A', 'B', 'C'], ['A', 'A', 'B', 'A', 'B', 'C', 'C', 'B', 'A', 'C'], ['C', 'A', 'C', 'B', 'C', 'A', 'A', 